In [1]:
import cv2
import numpy as np
from pynput.mouse import Button, Controller
import wx

In [2]:
lower_blue = np.array([110,100,100])
upper_blue = np.array([130,255,255])
lower_red = np.array([170,120,70])
upper_red = np.array([180,255,255])
mouse = Controller()

In [3]:
app=wx.App(False)
(sx,sy)=wx.GetDisplaySize()
(camx,camy)=(320,240)
kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))

In [ ]:
cap = cv2.VideoCapture(0)
pinchFlag = 0
while True:
    _, img=cap.read()
    img=cv2.resize(img,(camx,camy))

    imgHSV= cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
#     mask_red=cv2.inRange(imgHSV,lower_red,upper_red)
    mask_blue=cv2.inRange(imgHSV,lower_blue,upper_blue)

#     maskOpen_red=cv2.morphologyEx(mask_red,cv2.MORPH_OPEN,kernelOpen)
#     maskClose_red=cv2.morphologyEx(maskOpen_red,cv2.MORPH_CLOSE,kernelClose)
    
    maskOpen_blue=cv2.morphologyEx(mask_blue,cv2.MORPH_OPEN,kernelOpen)
    maskClose_blue=cv2.morphologyEx(maskOpen_blue,cv2.MORPH_CLOSE,kernelClose)

#     conts_red,h=cv2.findContours(maskClose_red.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    conts_blue,h=cv2.findContours(maskClose_blue.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)


    if(len(conts_blue)==2):
        if(pinchFlag==1):
            pinchFlag=0
            mouse.release(Button.left)
        hull1 = cv2.convexHull(conts_blue[0],False)
        img = cv2.drawContours(img,[hull1],-1,(0,0,255),1)
        M1 = cv2.moments(hull1)
        c1X = int(M1["m10"] / M1["m00"])
        c1Y = int(M1["m01"] / M1["m00"])
        
        hull2 = cv2.convexHull(conts_blue[1],False)
        img = cv2.drawContours(img,[hull2],-1,(0,0,255),1)
        M2 = cv2.moments(hull2)
        c2X = int(M2["m10"] / M2["m00"])
        c2Y = int(M2["m01"] / M2["m00"])
        
        cX = (c1X +c2X)/2
        cY = (c1Y +c2Y)/2

        mouseLoc=(sx-(cX*sx/camx), cY*sy/camy)
        mouse.position=mouseLoc 
        while mouse.position!=mouseLoc:
            pass
    elif(len(conts_blue)==1):
        if(pinchFlag==0):
            pinchFlag=1
            mouse.press(Button.left)
        hull = cv2.convexHull(conts_blue[0],False)
        img = cv2.drawContours(img,[hull],-1,(0,0,255),1)
        M = cv2.moments(hull)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        mouseLoc=(sx-(cX*sx/camx), cY*sy/camy)
        mouse.position=mouseLoc 
        while mouse.position!=mouseLoc:
            pass
    cv2.imshow("cam",img)
    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [ ]:
cap.release()